In [1]:
# -*- coding: UTF-8 -*-
import re
import jieba
import random
from tensorflow.contrib import learn

class Data_Prepare(object):

    def readfile(self, filename):
        texta = []
        textb = []
        tag = []
        with open(filename, 'r',encoding='utf-8') as f:
            for line in f.readlines():
                line = line.strip().split("\t")
                texta.append(self.pre_processing(line[0]))
                textb.append(self.pre_processing(line[1]))
                tag.append(line[2])
        # shuffle
        index = [x for x in range(len(texta))]
        random.shuffle(index)
        texta_new = [texta[x] for x in index]
        textb_new = [textb[x] for x in index]
        tag_new = [tag[x] for x in index]
        # 类别个数，构成[0,1]多分类向量
        type = list(set(tag_new))
        dicts = {}
        tags_vec = []
        for x in tag_new:
            if x not in dicts.keys():
                dicts[x] = 1
            else:
                dicts[x] += 1
            temp = [0] * len(type)
            temp[int(x)] = 1
            tags_vec.append(temp)
        print(dicts)
        return texta_new, textb_new, tags_vec

    def pre_processing(self, text):
        # 删除（）里的内容
        text = re.sub('（[^（.]*）', '', text)
        # 只保留中文部分
        text = ''.join([x for x in text if '\u4e00' <= x <= '\u9fa5'])
        # 利用jieba进行分词
        words = ' '.join(jieba.cut(text)).split(" ")
        # 不分词
        words = [x for x in ''.join(words)]
        return ' '.join(words)
    # 构建词汇表
    def build_vocab(self, sentences, path):
        # 每个词的长度，取得最大长度，作为一个句子的固定长度，不足的补0
        lens = [len(sentence.split(" ")) for sentence in sentences]
        max_length = max(lens)
        vocab_processor = learn.preprocessing.VocabularyProcessor(max_length)
        vocab_processor.fit(sentences)
        vocab_processor.save(path)


# -*- coding: UTF-8 -*-
class Config(object):

    def __init__(self):
        self.embedding_size = 50  # 词向量维度
        self.hidden_num = 100  # 隐藏层规模
        self.l2_lambda = 0.01
        self.learning_rate = 0.001
        self.dropout_keep_prob = 0.5
        self.attn_size = 200
        self.K = 2

        self.epoch = 10
        self.Batch_Size = 50
        
import tensorflow as tf
import math


class Bi_lstm(object):

    def __init__(self, is_trainning, seq_length, class_num, vocabulary_size, embedding_size, hidden_num,
                l2_lambda, learning_rate):
        self.is_trainning = is_trainning
        self.vocabulary_size = vocabulary_size
        self.embedding_size = embedding_size
        self.hidden_num = hidden_num
        self.seq_length = seq_length
        self.class_num = class_num

        # init placeholder
        self.text_a = tf.placeholder(tf.int32, [None, seq_length], name='text_a')
        self.text_b = tf.placeholder(tf.int32, [None, seq_length], name='text_b')
        self.y = tf.placeholder(tf.int32, [None, class_num], name='y')
        # real length
        self.a_length = tf.placeholder(tf.int32, [None], name='a_length')
        self.b_length = tf.placeholder(tf.int32, [None], name='b_length')
        self.dropout_keep_prob = tf.placeholder(tf.float32, name="dropout_keep_prob")

        # embedding层 论文中采用是预训练好的词向量 这里随机初始化一个词典 在训练过程中进行调整
        with tf.device('/cpu:0'), tf.name_scope("embedding"):
            self.vocab_matrix = tf.Variable(tf.truncated_normal(shape=[vocabulary_size, embedding_size],
                                                                stddev=1.0 / math.sqrt(embedding_size)),
                                            name='vacab_matrix')
            self.text_a_embed = tf.nn.embedding_lookup(self.vocab_matrix, self.text_a)
            self.text_b_embed = tf.nn.embedding_lookup(self.vocab_matrix, self.text_b)

        with tf.name_scope('Input_Encoding'):
            a = self.biLSTMBlock(self.text_a_embed, hidden_num, 'Input_Encoding/biLSTM', self.a_length)
            b = self.biLSTMBlock(self.text_b_embed, hidden_num, 'Input_Encoding/biLSTM', self.b_length, isreuse=True)
        # a-b
        diff = tf.subtract(a, b)

        with tf.name_scope("output"):
            initializer = tf.random_normal_initializer(0.0, 0.1)
            with tf.variable_scope('feed_foward'):
                outputs = tf.nn.dropout(diff, self.dropout_keep_prob)
                outputs = tf.reshape(outputs, [-1, hidden_num * seq_length * 2])
                outputs = tf.layers.dense(outputs, hidden_num, tf.nn.relu, kernel_initializer=initializer)
                self.logits = tf.layers.dense(outputs, class_num, tf.nn.tanh, kernel_initializer=initializer)
            self.score = tf.nn.softmax(self.logits, name='score')
            self.prediction = tf.argmax(self.score, 1, name="prediction")

        with tf.name_scope('cost'):
            self.cost = tf.nn.softmax_cross_entropy_with_logits_v2(labels=self.y, logits=self.logits)
            self.cost = tf.reduce_mean(self.cost)
            weights = [v for v in tf.trainable_variables() if ('w' in v.name) or ('kernel' in v.name)]
            l2_loss = tf.add_n([tf.nn.l2_loss(w) for w in weights]) * l2_lambda
            self.loss = l2_loss + self.cost

        self.accuracy = tf.reduce_mean(
            tf.cast(tf.equal(tf.argmax(self.y, axis=1), self.prediction), tf.float32))

        if not is_trainning:
            return

        tvars = tf.trainable_variables()
        grads, _ = tf.clip_by_global_norm(tf.gradients(self.loss, tvars), 5)

        optimizer = tf.train.AdamOptimizer(learning_rate)
        self.train_op = optimizer.apply_gradients(zip(grads, tvars))

    def biLSTMBlock(self, inputs, num_units, scope, seq_len=None, isreuse=False):
        with tf.variable_scope(scope, reuse=isreuse):
            cell_fw = tf.nn.rnn_cell.BasicLSTMCell(num_units)
            fw_lstm_cell = tf.contrib.rnn.DropoutWrapper(cell_fw, output_keep_prob=self.dropout_keep_prob)
            cell_bw = tf.nn.rnn_cell.BasicLSTMCell(num_units)
            bw_lstm_cell = tf.contrib.rnn.DropoutWrapper(cell_bw, output_keep_prob=self.dropout_keep_prob)

            (a_outputs, a_output_states) = tf.nn.bidirectional_dynamic_rnn(fw_lstm_cell, bw_lstm_cell,
                                                                           inputs,
                                                                           sequence_length=seq_len,
                                                                           dtype=tf.float32)
            a_bar = tf.concat(a_outputs, axis=2)
            return a_bar


import tensorflow as tf
import math


class Bi_lstm_Atten(object):

    def __init__(self, is_trainning, seq_length, class_num, vocabulary_size, embedding_size, hidden_num,
                 attn_size, l2_lambda, learning_rate):
        self.is_trainning = is_trainning
        self.vocabulary_size = vocabulary_size
        self.embedding_size = embedding_size
        self.hidden_num = hidden_num
        self.attn_size = attn_size
        self.seq_length = seq_length
        self.class_num = class_num

        # init placeholder
        self.text_a = tf.placeholder(tf.int32, [None, seq_length], name='text_a')
        self.text_b = tf.placeholder(tf.int32, [None, seq_length], name='text_b')
        self.y = tf.placeholder(tf.int32, [None, class_num], name='y')
        # real length
        self.a_length = tf.placeholder(tf.int32, [None], name='a_length')
        self.b_length = tf.placeholder(tf.int32, [None], name='b_length')
        self.dropout_keep_prob = tf.placeholder(tf.float32, name="dropout_keep_prob")

        # embedding层 论文中采用是预训练好的词向量 这里随机初始化一个词典 在训练过程中进行调整
        with tf.device('/cpu:0'), tf.name_scope("embedding"):
            self.vocab_matrix = tf.Variable(tf.truncated_normal(shape=[vocabulary_size, embedding_size],
                                                                stddev=1.0 / math.sqrt(embedding_size)),
                                            name='vacab_matrix')
            self.text_a_embed = tf.nn.embedding_lookup(self.vocab_matrix, self.text_a)
            self.text_b_embed = tf.nn.embedding_lookup(self.vocab_matrix, self.text_b)

        with tf.name_scope('Input_Encoding'):
            a = self.biLSTMBlock(self.text_a_embed, hidden_num, 'Input_Encoding/biLSTM', self.a_length)
            a_atten = self.attention(self.attn_size, a, hidden_num, seq_length, self.a_length)
            b = self.biLSTMBlock(self.text_b_embed, hidden_num, 'Input_Encoding/biLSTM', self.b_length, isreuse=True)
            b_atten = self.attention(self.attn_size, b, hidden_num, seq_length, self.b_length)

        diff = tf.subtract(a_atten, b_atten)

        with tf.name_scope("output"):
            initializer = tf.random_normal_initializer(0.0, 0.1)
            with tf.variable_scope('feed_foward'):
                outputs = tf.nn.dropout(diff, self.dropout_keep_prob)
                self.logits = tf.layers.dense(outputs, class_num, tf.nn.tanh, kernel_initializer=initializer)
            self.score = tf.nn.softmax(self.logits, name='score')
            self.prediction = tf.argmax(self.score, 1, name="prediction")

        with tf.name_scope('cost'):
            self.cost = tf.nn.softmax_cross_entropy_with_logits_v2(labels=self.y, logits=self.logits)
            self.cost = tf.reduce_mean(self.cost)
            weights = [v for v in tf.trainable_variables() if ('w' in v.name) or ('kernel' in v.name)]
            l2_loss = tf.add_n([tf.nn.l2_loss(w) for w in weights]) * l2_lambda
            self.loss = l2_loss + self.cost

        self.accuracy = tf.reduce_mean(
            tf.cast(tf.equal(tf.argmax(self.y, axis=1), self.prediction), tf.float32))

        if not is_trainning:
            return

        tvars = tf.trainable_variables()
        grads, _ = tf.clip_by_global_norm(tf.gradients(self.loss, tvars), 5)

        optimizer = tf.train.AdamOptimizer(learning_rate)
        self.train_op = optimizer.apply_gradients(zip(grads, tvars))

    def biLSTMBlock(self, inputs, num_units, scope, seq_len=None, isreuse=False):
        with tf.variable_scope(scope, reuse=isreuse):
            cell_fw = tf.nn.rnn_cell.BasicLSTMCell(num_units)
            fw_lstm_cell = tf.contrib.rnn.DropoutWrapper(cell_fw, output_keep_prob=self.dropout_keep_prob)
            cell_bw = tf.nn.rnn_cell.BasicLSTMCell(num_units)
            bw_lstm_cell = tf.contrib.rnn.DropoutWrapper(cell_bw, output_keep_prob=self.dropout_keep_prob)

            (a_outputs, a_output_states) = tf.nn.bidirectional_dynamic_rnn(fw_lstm_cell, bw_lstm_cell,
                                                                           inputs,
                                                                           sequence_length=seq_len,
                                                                           dtype=tf.float32)
            a_bar = tf.concat(a_outputs, axis=2)
            return a_bar

    def attention(self, attn_size, outputs, hidden_num, max_length, length):
        # attention
        attention_size = attn_size
        outputs = tf.transpose(outputs, [1, 0, 2])
        with tf.name_scope('attention'), tf.variable_scope('attention'):
            attention_w = tf.Variable(tf.truncated_normal([2 * hidden_num, attention_size], stddev=0.1),
                                      name='attention_w')
            attention_b = tf.Variable(tf.constant(0.1, shape=[attention_size]), name='attention_b')
            u_list = []
            for t in range(max_length):
                u_t = tf.tanh(tf.matmul(outputs[t], attention_w) + attention_b)
                u_list.append(u_t)
            u_w = tf.Variable(tf.truncated_normal([attention_size, 1], stddev=0.1), name='attention_uw')
            attn_z = []
            for t in range(max_length):
                z_t = tf.matmul(u_list[t], u_w)
                attn_z.append(z_t)
            # transform to batch_size * sequence_length
            attn_zconcat = tf.concat(attn_z, axis=1)
            # masked
            attn_zconcat = self.mask(attn_zconcat, length, max_length)
            self.alpha = tf.nn.softmax(attn_zconcat)
            # transform to sequence_length * batch_size * 1 , same rank as outputs
            alpha_trans = tf.expand_dims(self.alpha, -1)

        final_output = tf.reduce_sum(tf.transpose(outputs, [1, 0, 2]) * alpha_trans, 1)
        return final_output

    def mask(self, inputs, seq_len, max_len):
        mask = tf.cast(tf.sequence_mask(seq_len, maxlen=max_len), tf.float32)
        return inputs - (1 - mask) * 1e12


    
import tensorflow as tf
import data_prepare
from tensorflow.contrib import learn
import numpy as np
import bi_lstm
import config as config
from tqdm import tqdm
from sklearn.metrics import f1_score
from sklearn import metrics
import os

con = config.Config()
parent_path = os.path.dirname(os.getcwd())
data_pre = data_prepare.Data_Prepare()


class TrainModel(object):
    '''
        训练模型
        保存模型
    '''
    def pre_processing(self):

        train_texta, train_textb, train_tag = data_pre.readfile('data/train.txt')
        data = []
        data.extend(train_texta)
        data.extend(train_textb)
        # 训练集构造词典，并且保存
        data_pre.build_vocab(data, 'save_model/vocab/vocab.pickle')
        # 加载词典
        self.vocab_processor = learn.preprocessing.VocabularyProcessor.restore('save_model/vocab/vocab.pickle')
        # 中文句子，转化为id序列，固定长度
        train_texta_embedding = np.array(list(self.vocab_processor.transform(train_texta)))
        train_textb_embedding = np.array(list(self.vocab_processor.transform(train_textb)))
        print('train_texta_embedding:', train_texta_embedding[:5])
        print('train_textb_embedding:', train_textb_embedding[:5])


        dev_texta, dev_textb, dev_tag = data_pre.readfile('data/dev.txt')
        dev_texta_embedding = np.array(list(self.vocab_processor.transform(dev_texta)))
        dev_textb_embedding = np.array(list(self.vocab_processor.transform(dev_textb)))
        return train_texta_embedding, train_textb_embedding, np.array(train_tag), \
               dev_texta_embedding, dev_textb_embedding, np.array(dev_tag)

    def get_batches(self, texta, textb, tag):
        num_batch = int(len(texta) / con.Batch_Size)
        for i in range(num_batch):
            a = texta[i*con.Batch_Size:(i+1)*con.Batch_Size]
            b = textb[i*con.Batch_Size:(i+1)*con.Batch_Size]
            t = tag[i*con.Batch_Size:(i+1)*con.Batch_Size]
            yield a, b, t

    def get_length(self, trainX_batch):
        # sentence length
        lengths = []
        for sample in trainX_batch:
            count = 0
            for index in sample:
                if index != 0:
                    count += 1
                else:
                    break
            lengths.append(count)
        return lengths

    def trainModel(self):
        train_texta_embedding, train_textb_embedding, train_tag, \
        dev_texta_embedding, dev_textb_embedding, dev_tag = self.pre_processing()
        # 定义训练用的循环神经网络模型
        with tf.variable_scope('bi_listm_model', reuse=None):
            # bi_listm
            model = Bi_lstm(True, seq_length=len(train_texta_embedding[0]),
                                    class_num=len(train_tag[0]),
                                    vocabulary_size=len(self.vocab_processor.vocabulary_),
                                    embedding_size=con.embedding_size,
                                    hidden_num=con.hidden_num,
                                    l2_lambda=con.l2_lambda,
                                    learning_rate=con.learning_rate)

        # 训练模型
        with tf.Session() as sess:
            tf.global_variables_initializer().run()
            saver = tf.train.Saver()
            best_f1 = 0.0
            for time in range(con.epoch):
                print("training " + str(time + 1) + ">>>>>>>>>>>>>>>>>>>>>>>>>>>>>>")
                model.is_trainning = True
                loss_all = []
                accuracy_all = []
                for texta, textb, tag in tqdm(self.get_batches(train_texta_embedding, train_textb_embedding, train_tag)):
                    feed_dict = {
                        model.text_a: texta,
                        model.text_b: textb,
                        model.y: tag,
                        model.dropout_keep_prob: con.dropout_keep_prob,
                        model.a_length: np.array(self.get_length(texta)),
                        model.b_length: np.array(self.get_length(textb))
                    }
                    _, cost, accuracy = sess.run([model.train_op, model.loss, model.accuracy], feed_dict)
                    loss_all.append(cost)
                    accuracy_all.append(accuracy)

                print("第" + str((time + 1)) + "次迭代的损失为：" + str(np.mean(np.array(loss_all))) + ";准确率为：" +
                      str(np.mean(np.array(accuracy_all))))

                def dev_step():
                    """
                    Evaluates model on a dev set
                    """
                    loss_all = []
                    accuracy_all = []
                    predictions = []
                    for texta, textb, tag in tqdm(self.get_batches(dev_texta_embedding, dev_textb_embedding, dev_tag)):
                        feed_dict = {
                            model.text_a: texta,
                            model.text_b: textb,
                            model.y: tag,
                            model.dropout_keep_prob: 1.0,
                            model.a_length: np.array(self.get_length(texta)),
                            model.b_length: np.array(self.get_length(textb))
                        }
                        dev_cost, dev_accuracy, prediction = sess.run([model.loss, model.accuracy,
                                                                       model.prediction], feed_dict)
                        loss_all.append(dev_cost)
                        accuracy_all.append(dev_accuracy)
                        predictions.extend(prediction)
                    y_true = [np.nonzero(x)[0][0] for x in dev_tag]
                    y_true = y_true[0:len(loss_all)*con.Batch_Size]
                    f1 = f1_score(np.array(y_true), np.array(predictions), average='weighted')
                    print('分类报告:\n', metrics.classification_report(np.array(y_true), predictions))
                    print("验证集：loss {:g}, acc {:g}, f1 {:g}\n".format(np.mean(np.array(loss_all)),
                                                                      np.mean(np.array(accuracy_all)), f1))
                    return f1

                model.is_trainning = False
                f1 = dev_step()

                if f1 > best_f1:
                    best_f1 = f1
                    saver.save(sess, "save_model/ckpt/model.ckpt")
                    print("Saved model success\n")


if __name__ == '__main__':
    train = TrainModel()
    train.trainModel()

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\nie\AppData\Local\Temp\jieba.cache
Loading model cost 0.630 seconds.
Prefix dict has been built succesfully.


{'1': 138574, '0': 100192}
Instructions for updating:
Please use tensorflow/transform or tf.data.
Instructions for updating:
Please use tensorflow/transform or tf.data.
Instructions for updating:
Please use tensorflow/transform or tf.data.
train_texta_embedding: [[ 1  2  3  4  5  6  7  8  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0]
 [ 9 10 11 12 13 14 15 16 17 18 19 20 21 18  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0]
 [22 23 24 25 26 27 28 29 30  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0]
 [31 32 33 34 18 35 36  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0]
 [37 38 39 40 41 26 42 43  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0]]
train_textb_embedding: [[  5   6   7   8   1 232   3   4   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  9  10  11  12 

4775it [05:03, 15.89it/s]


第1次迭代的损失为：1.1273893;准确率为：0.72770256


176it [00:03, 44.61it/s]


...

分类报告:
               precision    recall  f1-score   support

           0       0.72      0.58      0.65      4399
           1       0.65      0.78      0.71      4401

   micro avg       0.68      0.68      0.68      8800
   macro avg       0.69      0.68      0.68      8800
weighted avg       0.69      0.68      0.68      8800

验证集：loss 0.701377, acc 0.680568, f1 0.677502

training 10>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>


4775it [04:44, 16.81it/s]


第10次迭代的损失为：0.50991726;准确率为：0.8337634


176it [00:04, 43.36it/s]


分类报告:
               precision    recall  f1-score   support

           0       0.72      0.59      0.65      4399
           1       0.65      0.77      0.71      4401

   micro avg       0.68      0.68      0.68      8800
   macro avg       0.69      0.68      0.68      8800
weighted avg       0.69      0.68      0.68      8800

验证集：loss 0.70274, acc 0.681932, f1 0.6794

Saved model success



In [1]:
# -*- coding: UTF-8 -*-
import re
import jieba
import random
from tensorflow.contrib import learn

class Data_Prepare(object):

    def readfile(self, filename):
        texta = []
        textb = []
        tag = []
        with open(filename, 'r',encoding='utf-8') as f:
            for line in f.readlines():
                line = line.strip().split("\t")
                texta.append(self.pre_processing(line[0]))
                textb.append(self.pre_processing(line[1]))
                tag.append(line[2])
        # shuffle
        index = [x for x in range(len(texta))]
        random.shuffle(index)
        texta_new = [texta[x] for x in index]
        textb_new = [textb[x] for x in index]
        tag_new = [tag[x] for x in index]
        # 类别个数，构成[0,1]多分类向量
        type = list(set(tag_new))
        dicts = {}
        tags_vec = []
        for x in tag_new:
            if x not in dicts.keys():
                dicts[x] = 1
            else:
                dicts[x] += 1
            temp = [0] * len(type)
            temp[int(x)] = 1
            tags_vec.append(temp)
        print(dicts)
        return texta_new, textb_new, tags_vec

    def pre_processing(self, text):
        # 删除（）里的内容
        text = re.sub('（[^（.]*）', '', text)
        # 只保留中文部分
        text = ''.join([x for x in text if '\u4e00' <= x <= '\u9fa5'])
        # 利用jieba进行分词
        words = ' '.join(jieba.cut(text)).split(" ")
        # 不分词
        words = [x for x in ''.join(words)]
        return ' '.join(words)
    # 构建词汇表
    def build_vocab(self, sentences, path):
        # 每个词的长度，取得最大长度，作为一个句子的固定长度，不足的补0
        lens = [len(sentence.split(" ")) for sentence in sentences]
        max_length = max(lens)
        vocab_processor = learn.preprocessing.VocabularyProcessor(max_length)
        vocab_processor.fit(sentences)
        vocab_processor.save(path)


# -*- coding: UTF-8 -*-
class Config(object):

    def __init__(self):
        self.embedding_size = 50  # 词向量维度
        self.hidden_num = 100  # 隐藏层规模
        self.l2_lambda = 0.01
        self.learning_rate = 0.001
        self.dropout_keep_prob = 0.5
        self.attn_size = 200
        self.K = 2

        self.epoch = 10
        self.Batch_Size = 50
        

import tensorflow as tf
import math


class Bi_lstm_Atten(object):

    def __init__(self, is_trainning, seq_length, class_num, vocabulary_size, embedding_size, hidden_num,
                 attn_size, l2_lambda, learning_rate):
        self.is_trainning = is_trainning
        self.vocabulary_size = vocabulary_size
        self.embedding_size = embedding_size
        self.hidden_num = hidden_num
        self.attn_size = attn_size
        self.seq_length = seq_length
        self.class_num = class_num

        # init placeholder
        self.text_a = tf.placeholder(tf.int32, [None, seq_length], name='text_a')
        self.text_b = tf.placeholder(tf.int32, [None, seq_length], name='text_b')
        self.y = tf.placeholder(tf.int32, [None, class_num], name='y')
        # real length
        self.a_length = tf.placeholder(tf.int32, [None], name='a_length')
        self.b_length = tf.placeholder(tf.int32, [None], name='b_length')
        self.dropout_keep_prob = tf.placeholder(tf.float32, name="dropout_keep_prob")

        # embedding层 论文中采用是预训练好的词向量 这里随机初始化一个词典 在训练过程中进行调整
        with tf.device('/cpu:0'), tf.name_scope("embedding"):
            self.vocab_matrix = tf.Variable(tf.truncated_normal(shape=[vocabulary_size, embedding_size],
                                                                stddev=1.0 / math.sqrt(embedding_size)),
                                            name='vacab_matrix')
            self.text_a_embed = tf.nn.embedding_lookup(self.vocab_matrix, self.text_a)
            self.text_b_embed = tf.nn.embedding_lookup(self.vocab_matrix, self.text_b)

        with tf.name_scope('Input_Encoding'):
            a = self.biLSTMBlock(self.text_a_embed, hidden_num, 'Input_Encoding/biLSTM', self.a_length)
            # 加了attention
            a_atten = self.attention(self.attn_size, a, hidden_num, seq_length, self.a_length)
            b = self.biLSTMBlock(self.text_b_embed, hidden_num, 'Input_Encoding/biLSTM', self.b_length, isreuse=True)
            b_atten = self.attention(self.attn_size, b, hidden_num, seq_length, self.b_length)

        diff = tf.subtract(a_atten, b_atten)

        with tf.name_scope("output"):
            initializer = tf.random_normal_initializer(0.0, 0.1)
            with tf.variable_scope('feed_foward'):
                outputs = tf.nn.dropout(diff, self.dropout_keep_prob)
                self.logits = tf.layers.dense(outputs, class_num, tf.nn.tanh, kernel_initializer=initializer)
            self.score = tf.nn.softmax(self.logits, name='score')
            self.prediction = tf.argmax(self.score, 1, name="prediction")

        with tf.name_scope('cost'):
            self.cost = tf.nn.softmax_cross_entropy_with_logits_v2(labels=self.y, logits=self.logits)
            self.cost = tf.reduce_mean(self.cost)
            weights = [v for v in tf.trainable_variables() if ('w' in v.name) or ('kernel' in v.name)]
            l2_loss = tf.add_n([tf.nn.l2_loss(w) for w in weights]) * l2_lambda
            self.loss = l2_loss + self.cost

        self.accuracy = tf.reduce_mean(
            tf.cast(tf.equal(tf.argmax(self.y, axis=1), self.prediction), tf.float32))

        if not is_trainning:
            return

        tvars = tf.trainable_variables()
        grads, _ = tf.clip_by_global_norm(tf.gradients(self.loss, tvars), 5)

        optimizer = tf.train.AdamOptimizer(learning_rate)
        self.train_op = optimizer.apply_gradients(zip(grads, tvars))

    def biLSTMBlock(self, inputs, num_units, scope, seq_len=None, isreuse=False):
        with tf.variable_scope(scope, reuse=isreuse):
            cell_fw = tf.nn.rnn_cell.BasicLSTMCell(num_units)
            fw_lstm_cell = tf.contrib.rnn.DropoutWrapper(cell_fw, output_keep_prob=self.dropout_keep_prob)
            cell_bw = tf.nn.rnn_cell.BasicLSTMCell(num_units)
            bw_lstm_cell = tf.contrib.rnn.DropoutWrapper(cell_bw, output_keep_prob=self.dropout_keep_prob)

            (a_outputs, a_output_states) = tf.nn.bidirectional_dynamic_rnn(fw_lstm_cell, bw_lstm_cell,
                                                                           inputs,
                                                                           sequence_length=seq_len,
                                                                           dtype=tf.float32)
            a_bar = tf.concat(a_outputs, axis=2)
            return a_bar

    def attention(self, attn_size, outputs, hidden_num, max_length, length):
        # attention
        attention_size = attn_size
        outputs = tf.transpose(outputs, [1, 0, 2])
        with tf.name_scope('attention'), tf.variable_scope('attention'):
            attention_w = tf.Variable(tf.truncated_normal([2 * hidden_num, attention_size], stddev=0.1),
                                      name='attention_w')
            attention_b = tf.Variable(tf.constant(0.1, shape=[attention_size]), name='attention_b')
            u_list = []
            for t in range(max_length):
                u_t = tf.tanh(tf.matmul(outputs[t], attention_w) + attention_b)
                u_list.append(u_t)
            u_w = tf.Variable(tf.truncated_normal([attention_size, 1], stddev=0.1), name='attention_uw')
            attn_z = []
            for t in range(max_length):
                z_t = tf.matmul(u_list[t], u_w)
                attn_z.append(z_t)
            # transform to batch_size * sequence_length
            attn_zconcat = tf.concat(attn_z, axis=1)
            # masked
            attn_zconcat = self.mask(attn_zconcat, length, max_length)
            self.alpha = tf.nn.softmax(attn_zconcat)
            # transform to sequence_length * batch_size * 1 , same rank as outputs
            alpha_trans = tf.expand_dims(self.alpha, -1)

        final_output = tf.reduce_sum(tf.transpose(outputs, [1, 0, 2]) * alpha_trans, 1)
        return final_output

    def mask(self, inputs, seq_len, max_len):
        mask = tf.cast(tf.sequence_mask(seq_len, maxlen=max_len), tf.float32)
        return inputs - (1 - mask) * 1e12


    
import tensorflow as tf
import data_prepare
from tensorflow.contrib import learn
import numpy as np
import bi_lstm
import config as config
from tqdm import tqdm
from sklearn.metrics import f1_score
from sklearn import metrics
import os

con = config.Config()
parent_path = os.path.dirname(os.getcwd())
data_pre = data_prepare.Data_Prepare()


class TrainModel(object):
    '''
        训练模型
        保存模型
    '''
    def pre_processing(self):

        train_texta, train_textb, train_tag = data_pre.readfile('data/train.txt')
        data = []
        data.extend(train_texta)
        data.extend(train_textb)
        # 训练集构造词典，并且保存
        data_pre.build_vocab(data, 'save_model/vocab/vocab.pickle')
        # 加载词典
        self.vocab_processor = learn.preprocessing.VocabularyProcessor.restore('save_model/vocab/vocab.pickle')
        # 中文句子，转化为id序列，固定长度
        train_texta_embedding = np.array(list(self.vocab_processor.transform(train_texta)))
        train_textb_embedding = np.array(list(self.vocab_processor.transform(train_textb)))
        print('train_texta_embedding:', train_texta_embedding[:5])
        print('train_textb_embedding:', train_textb_embedding[:5])


        dev_texta, dev_textb, dev_tag = data_pre.readfile('data/dev.txt')
        dev_texta_embedding = np.array(list(self.vocab_processor.transform(dev_texta)))
        dev_textb_embedding = np.array(list(self.vocab_processor.transform(dev_textb)))
        return train_texta_embedding, train_textb_embedding, np.array(train_tag), \
               dev_texta_embedding, dev_textb_embedding, np.array(dev_tag)

    def get_batches(self, texta, textb, tag):
        num_batch = int(len(texta) / con.Batch_Size)
        for i in range(num_batch):
            a = texta[i*con.Batch_Size:(i+1)*con.Batch_Size]
            b = textb[i*con.Batch_Size:(i+1)*con.Batch_Size]
            t = tag[i*con.Batch_Size:(i+1)*con.Batch_Size]
            yield a, b, t

    def get_length(self, trainX_batch):
        # sentence length
        lengths = []
        for sample in trainX_batch:
            count = 0
            for index in sample:
                if index != 0:
                    count += 1
                else:
                    break
            lengths.append(count)
        return lengths

    def trainModel(self):
        train_texta_embedding, train_textb_embedding, train_tag, \
        dev_texta_embedding, dev_textb_embedding, dev_tag = self.pre_processing()
        # 定义训练用的循环神经网络模型
        with tf.variable_scope('bi_listm_model', reuse=None):
            # bi_listm_attn
            model = Bi_lstm_Atten(True, seq_length=len(train_texta_embedding[0]),
                                    class_num=len(train_tag[0]),
                                    vocabulary_size=len(self.vocab_processor.vocabulary_),
                                    embedding_size=con.embedding_size,
                                    hidden_num=con.hidden_num,
                                    attn_size = con.attn_size,
                                    l2_lambda=con.l2_lambda,
                                    learning_rate=con.learning_rate)

        # 训练模型
        with tf.Session() as sess:
            tf.global_variables_initializer().run()
            saver = tf.train.Saver()
            best_f1 = 0.0
            for time in range(con.epoch):
                print("training " + str(time + 1) + ">>>>>>>>>>>>>>>>>>>>>>>>>>>>>>")
                model.is_trainning = True
                loss_all = []
                accuracy_all = []
                for texta, textb, tag in tqdm(self.get_batches(train_texta_embedding, train_textb_embedding, train_tag)):
                    feed_dict = {
                        model.text_a: texta,
                        model.text_b: textb,
                        model.y: tag,
                        model.dropout_keep_prob: con.dropout_keep_prob,
                        model.a_length: np.array(self.get_length(texta)),
                        model.b_length: np.array(self.get_length(textb))
                    }
                    _, cost, accuracy = sess.run([model.train_op, model.loss, model.accuracy], feed_dict)
                    loss_all.append(cost)
                    accuracy_all.append(accuracy)

                print("第" + str((time + 1)) + "次迭代的损失为：" + str(np.mean(np.array(loss_all))) + ";准确率为：" +
                      str(np.mean(np.array(accuracy_all))))

                def dev_step():
                    """
                    Evaluates model on a dev set
                    """
                    loss_all = []
                    accuracy_all = []
                    predictions = []
                    for texta, textb, tag in tqdm(self.get_batches(dev_texta_embedding, dev_textb_embedding, dev_tag)):
                        feed_dict = {
                            model.text_a: texta,
                            model.text_b: textb,
                            model.y: tag,
                            model.dropout_keep_prob: 1.0,
                            model.a_length: np.array(self.get_length(texta)),
                            model.b_length: np.array(self.get_length(textb))
                        }
                        dev_cost, dev_accuracy, prediction = sess.run([model.loss, model.accuracy,
                                                                       model.prediction], feed_dict)
                        loss_all.append(dev_cost)
                        accuracy_all.append(dev_accuracy)
                        predictions.extend(prediction)
                    y_true = [np.nonzero(x)[0][0] for x in dev_tag]
                    y_true = y_true[0:len(loss_all)*con.Batch_Size]
                    f1 = f1_score(np.array(y_true), np.array(predictions), average='weighted')
                    print('分类报告:\n', metrics.classification_report(np.array(y_true), predictions))
                    print("验证集：loss {:g}, acc {:g}, f1 {:g}\n".format(np.mean(np.array(loss_all)),
                                                                      np.mean(np.array(accuracy_all)), f1))
                    return f1

                model.is_trainning = False
                f1 = dev_step()

                if f1 > best_f1:
                    best_f1 = f1
                    saver.save(sess, "save_model/ckpt/model.ckpt")
                    print("Saved model success\n")


if __name__ == '__main__':
    train = TrainModel()
    train.trainModel()

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\nie\AppData\Local\Temp\jieba.cache
Loading model cost 2.232 seconds.
Prefix dict has been built succesfully.


{'1': 138574, '0': 100192}
Instructions for updating:
Please use tensorflow/transform or tf.data.
Instructions for updating:
Please use tensorflow/transform or tf.data.
Instructions for updating:
Please use tensorflow/transform or tf.data.
train_texta_embedding: [[ 1  2  3  4  5  6  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0]
 [ 7  8  9 10 11 12 13 14  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0]
 [15 16 17 18 19 20 21 22 23 24  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0]
 [25 26 27 28 29 30 31 32 33 34 35 20 21 35 36 37  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0]
 [38 39 40 41 21  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0]]
train_textb_embedding: [[   4    5    6   52    1    2  430    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0   

4775it [23:30,  3.14it/s]


第1次迭代的损失为：0.72725546;准确率为：0.5803979


176it [00:18,  9.46it/s]
e:\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
e:\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


...

分类报告:
               precision    recall  f1-score   support

           0       0.50      0.42      0.46      4399
           1       0.50      0.59      0.54      4401

   micro avg       0.50      0.50      0.50      8800
   macro avg       0.50      0.50      0.50      8800
weighted avg       0.50      0.50      0.50      8800

验证集：loss 0.812499, acc 0.503636, f1 0.500092

Saved model success

training 10>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>


4775it [20:36,  3.90it/s]


第10次迭代的损失为：0.64361686;准确率为：0.69863456


176it [00:12, 14.57it/s]


分类报告:
               precision    recall  f1-score   support

           0       0.51      0.42      0.46      4399
           1       0.51      0.61      0.55      4401

   micro avg       0.51      0.51      0.51      8800
   macro avg       0.51      0.51      0.51      8800
weighted avg       0.51      0.51      0.51      8800

验证集：loss 0.817204, acc 0.511932, f1 0.507357

Saved model success



In [2]:
import tensorflow.contrib.learn as learn
import tensorflow as tf
import numpy as np
import data_prepare

data_pre = data_prepare.Data_Prepare()


class Infer(object):
    """
        ues model to predict classification.
    """
    def __init__(self):
        self.vocab_processor = learn.preprocessing.VocabularyProcessor.restore('save_model/vocab/vocab.pickle')
        self.checkpoint_file = tf.train.latest_checkpoint('save_model/ckpt')
        graph = tf.Graph()
        with graph.as_default():
            session_conf = tf.ConfigProto(allow_soft_placement=True, log_device_placement=False)
            self.sess = tf.Session(config=session_conf)
            with self.sess.as_default():
                # Load the saved meta graph and restore variables
                saver = tf.train.import_meta_graph("{}.meta".format(self.checkpoint_file))
                saver.restore(self.sess, self.checkpoint_file)

                # Get the placeholders from the graph by name
                self.text_a = graph.get_operation_by_name("bi_listm_model/text_a").outputs[0]
                self.text_b = graph.get_operation_by_name("bi_listm_model/text_b").outputs[0]
                self.a_length = graph.get_operation_by_name("bi_listm_model/a_length").outputs[0]
                self.b_length = graph.get_operation_by_name("bi_listm_model/b_length").outputs[0]
                self.drop_keep_prob = graph.get_operation_by_name("bi_listm_model/dropout_keep_prob").outputs[0]

                # Tensors we want to evaluate
                self.prediction = graph.get_operation_by_name("bi_listm_model/output/prediction").outputs[0]
                self.score = graph.get_operation_by_name("bi_listm_model/output/score").outputs[0]

    def infer(self, sentenceA, sentenceB):
        # transfer to vector
        sentenceA = [data_pre.pre_processing(sentenceA)]
        sentenceB = [data_pre.pre_processing(sentenceB)]
        vector_A = np.array(list(self.vocab_processor.transform(sentenceA)))
        vector_B = np.array(list(self.vocab_processor.transform(sentenceB)))
        feed_dict = {
            self.text_a: vector_A,
            self.text_b: vector_B,
            self.drop_keep_prob: 1.0,
            self.a_length: np.array([len(sentenceA[0].split(" "))]),
            self.b_length: np.array([len(sentenceB[0].split(" "))])
        }
        y, s = self.sess.run([self.prediction, self.score], feed_dict)
        return y, s


if __name__ == '__main__':
    infer = Infer()
    sentencea = '你点击详情'
    sentenceb = '您点击详情'
    print(infer.infer(sentencea, sentenceb))

Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from save_model/ckpt\model.ckpt
(array([1], dtype=int64), array([[0.3601215, 0.6398785]], dtype=float32))
